# **Mount**

Mount the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install import-ipynb
%cd "/content/gdrive/MyDrive/progetto/code/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/progetto/code


# **Import**

Import the LSH_Index file,necessary to import the index

In [ ]:
import import_ipynb
import pickle
import numpy as np
from LSH_Index import *

importing Jupyter notebook from LSH_Index.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
'''
# Unzipping dataset
%cd '/content/gdrive/MyDrive/progetto/dbs/' 
!unzip -q total_dbs.zip
'''

# **Setup**

We setup the path of the ids and features numpy,saved at the end of the features extraction

In [ ]:
import numpy as np

finetuned_ids_path = '/content/gdrive/MyDrive/progetto/pickles/image_ids_finetuning.npy'
finetuned_features_path = '/content/gdrive/MyDrive/progetto/pickles/image_features_finetuning.npy'
#finetuned_features_path = '/content/gdrive/MyDrive/progetto/pickles/image_features_finetuning_new.npy'
finetuned_labels_path = '/content/gdrive/MyDrive/progetto/pickles/image_labels_finetuning.npy'

#prova con il fine tuning early stop
#finetuned_ids_path = '/content/gdrive/MyDrive/progetto/pickles/image_ids_finetuning_early_stop.npy'
#finetuned_features_path = '/content/gdrive/MyDrive/progetto/pickles/image_features_finetuning_early_stop.npy'
#finetuned_labels_path = '/content/gdrive/MyDrive/progetto/pickles/image_labels_finetuning_early_stop.npy'

ids_path = '/content/gdrive/MyDrive/progetto/pickles/image_ids_complete.npy'
features_path = '/content/gdrive/MyDrive/progetto/pickles/image_features_complete.npy'
labels_path = '/content/gdrive/MyDrive/progetto/pickles/image_labels_complete.npy'

# **Pickles**

Import the pickles, previously saved, belonging to the finetuned training set. In addition we compute the mean and standard deviation of the features, necessary to create the index

In [ ]:
import pickle

#numpy_ids = np.load(ids_path)
#numpy_features = np.load(features_path)
#numpy_labels = np.load(labels_path)

numpy_ids = np.load(finetuned_ids_path)
numpy_features = np.load(finetuned_features_path)
numpy_labels = np.load(finetuned_labels_path)

mean =  np.mean(numpy_features,axis=0)
std = np.std(numpy_features,axis=0)


# **Creation**

We create the index, specifying the number of g,h functions, the dimension of the features provided, the mean and standard deviation, and the type of search function we want to use. (commented lines are various tries). Each features belonging to the training set is inserted in the index. Finally, we print the time needed to create the index

In [ ]:
import time

print("creating index...")
startTime = time.perf_counter()
#LSHashIndex = LSHash(8,128,8,mean,std)
#LSHashIndex1 = LSHash(70,128,8,mean,std,"cosine")
#LSHashIndex = LSHash(32,128,32,mean,std)
#LSHashIndex1 = LSHash(32,128,32,mean,std,"cosine")
#LSHashIndex1 = LSHash(5,128,5,mean,std,"cosine")
LSHashIndex = LSHash(70,128,8,mean,std,"cosine")

for k in range(len(numpy_features)):
#for k in range(100):
  actual_feat = numpy_features[k]
  actual_id = numpy_ids[k]
  actual_label = numpy_labels[k]
  LSHashIndex.insert_features(actual_id,actual_feat,actual_label)
  #LSHashIndex1.insert_features(actual_id,actual_feat,actual_label)

print("index created in "+str(time.perf_counter()-startTime))


creating index...
index created in 555.6094054589998


# **Utility functions**

We define two functions, one necessary to compute the compute the sum of all the keys, the other one to compute the standard deviation (the first one corresponds to the total number of bucket created, the second one to the standard deviation between buckets)

In [ ]:
def retrieve_sumKeys(keys):
  sumKeys = np.zeros(7)
  for key in keys:
    for i in range(7):
      sumKeys[i] += key[i]
  return sumKeys 

def retrieve_std(l,i):
  array = []
  for bucket_g in l:
    bucket = bucket_g[i]
    for element in bucket:
      array.append(element)
  return np.std(array)

# **Performance**

We print, for each combination of tested g-h functions, the number of buckets, numer of total items in the buckets, average number of elements in each bucket, and the standard deviation between the buckets

In [ ]:

#Gs = [2,3,4,5,6,7,8]
#Gs = [2]
Gs = [2,3,4,5,6,7,8,16,30,32,40,50,60,70]
featuresCount = len(numpy_features)
print("num features: "+str(featuresCount))
for g in Gs:
   keys = LSHashIndex.return_keys_gFunction(g)
   l = LSHashIndex.return_bucketsDimByG(g)
   sumKeys = retrieve_sumKeys(keys)
   for i in range(7):
      std = retrieve_std(l,i)
      print("g function:" + str(g) + " , h function:" + str(i+2) + " , num Bucket:" + str(sumKeys[i]) + " , num items: " + str(featuresCount*g) + " , average per Bucket: " + str((featuresCount*g)/sumKeys[i]) + " , std per Bucket: " + str(std))

num features: 53709
g function:2 , h function:2 , num Bucket:881.0 , num items: 107418 , average per Bucket: 121.92735527809307 , std per Bucket: 270.9900743301519
g function:2 , h function:3 , num Bucket:4837.0 , num items: 107418 , average per Bucket: 22.20756667355799 , std per Bucket: 48.297941326518966
g function:2 , h function:4 , num Bucket:14962.0 , num items: 107418 , average per Bucket: 7.179387782382035 , std per Bucket: 14.26967814223012
g function:2 , h function:5 , num Bucket:35994.0 , num items: 107418 , average per Bucket: 2.9843307217869643 , std per Bucket: 4.717958354678509
g function:2 , h function:6 , num Bucket:62655.0 , num items: 107418 , average per Bucket: 1.7144361982283935 , std per Bucket: 2.0174534384745857
g function:2 , h function:7 , num Bucket:81732.0 , num items: 107418 , average per Bucket: 1.3142710321538686 , std per Bucket: 1.1091048064493847
g function:2 , h function:8 , num Bucket:93236.0 , num items: 107418 , average per Bucket: 1.1521086275687

# **Save**

We save the index ( or indexes, commented lines are various tries ) as a pickle in order to be later used

In [ ]:
print("Saving...")
pick_path = '/content/gdrive/MyDrive/progetto/pickles/lsh_models'
definitive ="/definitive/"
#picklefile = open(pick_path + definitive + "finetuned(5,5).pickle", 'wb')
#pickle.dump(LSHashIndex1,picklefile)
picklefile = open(pick_path + definitive + "finetuned(5,5).pickle", 'wb')
pickle.dump(LSHashIndex,picklefile)
picklefile.close()
'''
eucl = "/euclidean_distance/"
cosine = "/cosine_similarity/"
picklefile = open(pick_path + eucl + "complete(70,8)_early.pickle", 'wb')
pickle.dump(LSHashIndex, picklefile)
picklefile.close()
picklefile1 = open(pick_path + cosine + "complete(70,8)_early.pickle", 'wb')
pickle.dump(LSHashIndex1, picklefile1)
picklefile1.close()
'''
print("Saving finished")

Saving...
Saving finished
